GitHub:

Welcome to your assignment this week! 

To better understand bias and discrimination in AI, in this assignment, we will look at a Natural Language Processing use case. 

Natural Language Pocessing (NLP) is a branch of Artificial Intelligence (AI) that helps computers to understand, to interpret and to manipulate natural (i.e. human) language.
Imagine NLP-powered machines as black boxes that are capable of understanding and evaluating the context of the input documents (i.e. collection of words), outputting meaningful results that depend on the task the machine is designed for.


![](images/1_3zMvUnPzYZF9CSHdj6hT5A.png "the title")



<caption><center> Documents are fed into magic NLP model capable to get, for instance, the sentiment of the original content</center></caption>



Just like any other machine learning algorithm, biased data results in biased outcomes. And just like any other algorithm, results debiasing is painfully annoying, to the point that it might be simpler to unbias the society itself.

# The big deal: word embeddings

Words must be represented as **numeric vectors** in order to be fed into machine learning algorithms. One of the most powerful (and popular) ways to do it is through **Word Embeddings**. In word embedding models, each word in a given language is assigned to a high-dimensional vector, such that **the geometry of the vectors captures relations between the words**. 


Because word embeddings are very computionally expensive to train, most ML practitioners will load a pre-trained set of embeddings. 

**After this assignment you will be able to:**

- Load pre-trained word vectors, and measure similarity using cosine similarity
- Use word embeddings to solve word analogy problems such as Man is to Woman as King is to ______. 
- Modify word embeddings to reduce their gender bias 

Let's get started! Run the following cell to install all the packages you will need.

In [1]:
#!pip install numpy
#!pip install keras
#!pip install tensorflow

Run the following cell to load the packages you will need.

In [2]:
import numpy as np
import importlib
import pandas as pd
from IPython.display import display, Markdown
from w2v_utils import *

Using TensorFlow backend.
C:\Users\61431\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\61431\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\61431\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\61431\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

Next, lets load the word vectors. For this assignment, we will use 50-dimensional GloVe vectors to represent words. Run the following cell to load the `word_to_vec_map`. 

In [3]:
words, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

You've loaded:
- `words`: set of words in the vocabulary.
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.



In [4]:
print("Example of words: ",list(words)[:10])
print("Vector for word 'person' = ", word_to_vec_map.get('person'))

Example of words:  ['sikeston', 'jochimsen', 'endothelial', '51.85', 'wyke', 'banpais', 'sts-135', 'recollection', '12.44', 'ineffectual']
Vector for word 'person' =  [ 0.61734    0.40035    0.067786  -0.34263    2.0647     0.60844
  0.32558    0.3869     0.36906    0.16553    0.0065053 -0.075674
  0.57099    0.17314    1.0142    -0.49581   -0.38152    0.49255
 -0.16737   -0.33948   -0.44405    0.77543    0.20935    0.6007
  0.86649   -1.8923    -0.37901   -0.28044    0.64214   -0.23549
  2.9358    -0.086004  -0.14327   -0.50161    0.25291   -0.065446
  0.60768    0.13984    0.018135  -0.34877    0.039985   0.07943
  0.39318    1.0562    -0.23624   -0.4194    -0.35332   -0.15234
  0.62158    0.79257  ]


GloVe vectors provide much more useful information about the meaning of individual words. Lets now see how you can use GloVe vectors to decide how similar two words are. 


# Cosine similarity

To measure how similar two words are, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

\begin{equation}
\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}
\end{equation}

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 


![](images/cosine_sim.png)

<caption><center> <b>Figure 1</b>: The cosine of the angle between two vectors is a measure of how similar they are</center></caption>



***

**Task 1**: Implement the function `cosine_similarity()` to evaluate similarity between word vectors.

**Reminder**: The norm of $u$ is defined as $||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$

In [5]:
## Task 1
# cosine_similarity

def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    dot_product_uv = np.dot(u,v)               # Dot product of vectors u and v
    
    sq_u = np.square(u)                        # Taking square of each element in vector u
    sq_v = np.square(v)                        # Taking square of each element in vector v
    
    length_of_u = np.sqrt(np.sum(sq_u))        # Finding norm of vector u
    length_of_v= np.sqrt(np.sum(sq_v))         # Finding norm of vector v
    
    cosineSimilarity = dot_product_uv / (length_of_u * length_of_v)
    
    return cosineSimilarity

***

**Task 2**: Implement `most_similar_word` which returns the most similar word to a word.

In [6]:
## Task 2
# GRADED FUNCTION: most_similar_word

def most_similar_word(word, word_to_vec_map):
    """
    Most similar word return the most similar word to the word u.
        
    Arguments:
        word -- a word, string        
        word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the most similar word to u as measured by cosine similarity
    """
    # Word_to_vec_map is a dictionary where word string is a key and its vector is a vlaue. Lets iterate it and send it
    # to cosine_similarity() function to find most similar word to input 'word'
    
    u = word_to_vec_map.get(word)                           # get vector of 'word' from dictionary word_to_vec_map
    most_similar_word = ''
    max_cosineSimilarity_uv = 0
    for w in words:
        if w != word:                                       # So that exact same word are not sent for finding similarity
            v = word_to_vec_map.get(w)                      # get vector of word w from dictionary word_to_vec_map
            cosineSimilarity_uv = cosine_similarity(u,v)
            if cosineSimilarity_uv > max_cosineSimilarity_uv:
                max_cosineSimilarity_uv = cosineSimilarity_uv
                most_similar_word = w
    
    return most_similar_word
                




<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Some custom functions before moving ahead
</p>
</div>

In [7]:
# Custom function to check if words exists in dictionary before further calculation to handle not found error
def checkIfWordExistsInDictionary(w):                                          
    if w not in word_to_vec_map.keys():
        return False
    else:
        return True

# Defining custom print function (wrapper for ipython display)
def printp(string):
    colorstr = "<p style='color:{};font-weight:{};font-family:{}'>{}</p>".format('firebrick','bold','verdana', string)
    display(Markdown(colorstr))

def printg(string):
    colorstr = "<p style='color:{};font-weight:{};font-family:{}'>{}</p>".format('green','bold','monaco', string)
    display(Markdown(colorstr))

Answer the questions below:

**TASK 3: Write a code the answer the following questions:**

***

What is the similarity between the words brother and friend?

In [8]:
if checkIfWordExistsInDictionary('brother') and checkIfWordExistsInDictionary('friend'):
    u = word_to_vec_map.get('brother') 
    v = word_to_vec_map.get('friend')
    cosineSimilarity = cosine_similarity(u, v)
    printp('similarity between the words brother and friend is %f'%cosineSimilarity)
else:
    printp('One or all the words do not exists in the mapping dictionary. Check if the words are correctly spelled !!!')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>similarity between the words brother and friend is 0.871318</p>

What is the similarity between the words computer and kid?

In [9]:
if checkIfWordExistsInDictionary('computer') and checkIfWordExistsInDictionary('kid'):
    u = word_to_vec_map.get('computer') 
    v = word_to_vec_map.get('kid')
    cosineSimilarity = cosine_similarity(u, v)
    printp('similarity between the words computer and kid is %f'%cosineSimilarity)
else:
    printp('One or all the words do not exists in the mapping dictionary. Check if the words are correctly spelled')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>similarity between the words computer and kid is 0.438002</p>

What is the similarity between the words V1=(france - paris) and V2=(rome - italy)?

In [10]:
if checkIfWordExistsInDictionary('france') and checkIfWordExistsInDictionary('paris') and \
    checkIfWordExistsInDictionary('rome') and checkIfWordExistsInDictionary('italy'):
    u1 = word_to_vec_map.get('france') 
    u2 = word_to_vec_map.get('paris') 
    u3 = word_to_vec_map.get('rome')
    u4 = word_to_vec_map.get('italy') 
    
    v1 = np.subtract(u1, u2)
    v2 = np.subtract(u3, u4)
    
    cosineSimilarity = cosine_similarity(v1, v2)
    printp('similarity between the vectors V1 and V2 is = %f'%cosineSimilarity)
else:
    printp('One or all the words do not exists in the mapping dictionary. Check if the words are correctly spelled')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>similarity between the vectors V1 and V2 is = -0.675148</p>

What is the most similar word to computer?


In [11]:
if checkIfWordExistsInDictionary('computer'):
    w = 'computer'
    the_most_similar_word = most_similar_word(w, word_to_vec_map)
    printp('The most similar word to computer is - %s'%the_most_similar_word)
else:
    printp('One or all the words do not exists in the mapping dictionary. Check if the words are correctly spelled')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>The most similar word to computer is - computers</p>

What is the most similar word to australia?

In [12]:
if checkIfWordExistsInDictionary('australia'):
    u = 'australia'
    the_most_similar_word = most_similar_word(u, word_to_vec_map)
    printp('The most similar word to australia is - %s'%the_most_similar_word)
else:
    printp('One or all the words do not exists in the mapping dictionary. Check if the words are correctly spelled')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>The most similar word to australia is - zealand</p>

What is the most similar word to python?


In [13]:
if checkIfWordExistsInDictionary('python'):
    w = 'python'
    the_most_similar_word = most_similar_word(w, word_to_vec_map)
    printp('The most similar word to python is - %s'%the_most_similar_word)
else:
    printp('One or all the words do not exists in the mapping dictionary. Check if the words are correctly spelled')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>The most similar word to python is - reticulated</p>

***

Playing around the cosine similarity of other inputs will give you a better sense of how word vectors behave. 

#  Word analogy task

In the word analogy task, we complete the sentence <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. An example is <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. In detail, we are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. We will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity. 


**Task 4**: Complete the code below to be able to perform word analogies!

***

In [14]:
## Task 4
# GRADED FUNCTION: complete_analogy

def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    w_a = word_to_vec_map.get(word_a)                           # get vector of 'word_a' from dictionary word_to_vec_map
    w_b = word_to_vec_map.get(word_b)                           # get vector of 'word_b' from dictionary word_to_vec_map
    w_c = word_to_vec_map.get(word_c)                           # get vector of 'word_c' from dictionary word_to_vec_map
             
    most_similar_word_analogy = ''
    max_cosineSimilarity_uv = 0
    
    for w in words:
        if w != word_a and w != word_b and w != word_c:     # So that exact same word are not sent for finding similarity
            w_d = word_to_vec_map.get(w)                    # get vector of word w from dictionary word_to_vec_map
            v = np.subtract(w_c, w_d)
            u = np.subtract(w_a, w_b)
            cosineSimilarity_uv = cosine_similarity(u,v)
            if cosineSimilarity_uv > max_cosineSimilarity_uv:
                max_cosineSimilarity_uv = cosineSimilarity_uv
                most_similar_word_analogy = w

    return most_similar_word_analogy


Run the cell below to test your code, this may take 1-2 minutes.

In [15]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), \
                 ('small', 'smaller', 'large'), ('cardiologist','heart','dermatologist')]
for triad in triads_to_try:
    if checkIfWordExistsInDictionary(triad[0]) and checkIfWordExistsInDictionary(triad[1]) and checkIfWordExistsInDictionary(triad[2]):
        print('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(triad[0],triad[1], triad[2], word_to_vec_map)))
    else:
        printp('One or all the words do not exists in the mapping dictionary. Check if the words are correctly spelled')

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger
cardiologist -> heart :: dermatologist -> skin


<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Below are some challenging analogy triads for which our function fails
</p>
</div>

In [16]:
challenging_triads = [('child', 'human', 'poult'), ('muddled', 'confused', 'addled'), ('polio','virus','tuberculosis'), \
                      ('cardiologist','heart','chiropractor')]

for t in challenging_triads:
    if checkIfWordExistsInDictionary(t[0]) and checkIfWordExistsInDictionary(t[1]) and checkIfWordExistsInDictionary(t[2]):
        print ('{} -> {} :: {} -> {}'.format( *t, complete_analogy(t[0],t[1], t[2], word_to_vec_map)))
    else:
        printp('One or all the words in %s, %s, %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%(t[0],t[1],t[2]))

<p style='color:firebrick;font-weight:bold;font-family:verdana'>One or all the words in child, human, poult do not exists in the mapping dictionary. Check if the words are correctly spelled</p>

muddled -> confused :: addled -> young
polio -> virus :: tuberculosis -> viruses
cardiologist -> heart :: chiropractor -> broken


Once you get the correct expected output, please feel free to modify the input cells above to test your own analogies. Try to find some other analogy pairs that do work, but also find some where the algorithm doesn't give the right answer: For example, you can try small->smaller as big->?.  

# Debiasing word vectors

In the following exercise, you will examine gender biases that can be reflected in a word embedding, and explore algorithms for reducing the bias. In addition to learning about the topic of debiasing, this exercise will also help hone your intuition about what word vectors are doing. This section involves a bit of linear algebra, though you can probably complete it even without being expert in linear algebra, and we encourage you to give it a shot. 

Lets first see how the GloVe word embeddings relate to gender. You will first compute a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. The resulting vector $g$ roughly encodes the concept of "gender". (You might get a more accurate representation if you compute $g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl}-e_{boy}$, etc. and average over them. But just using $e_{woman}-e_{man}$ will give good enough results for now.) 


**Task 5:** Compute the bias vector using woman - man

***

In [17]:
u = []
v = []
g = []

w1 = 'woman'
w2 = 'man'
u = word_to_vec_map.get(w1) 
v = word_to_vec_map.get(w2) 
g = np.subtract(u, v)

printp('Below is the resultant bias vector of vector woman and vector man ')
print(g)

<p style='color:firebrick;font-weight:bold;font-family:verdana'>Below is the resultant bias vector of vector woman and vector man </p>

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


***

Now, you will consider the cosine similarity of different words with $g$. Consider what a positive value of similarity means vs a negative cosine similarity. 

**Task 6: Compute and print the similarity between g and the words in name_list**

***
    

In [18]:
df = pd.DataFrame(columns = ['Word','Cosine_similarity_with_g'])

name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for n in name_list:
    if checkIfWordExistsInDictionary(n):
        u1 = word_to_vec_map.get(n) 
        cosine_similarity_score= cosine_similarity(u1, g)
        df = df.append({'Word':n, 'Cosine_similarity_with_g':cosine_similarity_score}, ignore_index=True)
    else:
        printp('One or all the words in %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%n)
printp('List of names and their similarities with constructed vector g:')
display(df)

<p style='color:firebrick;font-weight:bold;font-family:verdana'>List of names and their similarities with constructed vector g:</p>

,Word,Cosine_similarity_with_g
0,john,-0.231634
1,marie,0.315598
2,sophie,0.318688
3,ronaldo,-0.312448
4,priya,0.176320
5,rahul,-0.169155
6,danielle,0.243933
7,reza,-0.079304
8,katy,0.283107
9,yasmin,0.233139


<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Lets change subtraction sequence and try ... <br>
g = man - woman
</p>
</div>

In [19]:
u = []
v = []
g = []

w1 = 'woman'
w2 = 'man'

u = word_to_vec_map.get(w1) 
v = word_to_vec_map.get(w2) 
g = np.subtract(v, u)

printp('Below is the resultant bias vector of vector man and vector woman ')
print(g)

df = pd.DataFrame(columns = ['Word','Cosine_similarity_with_g'])

name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for n in name_list:
    if checkIfWordExistsInDictionary(n):
        u1 = word_to_vec_map.get(n) 
        cosine_similarity_score= cosine_similarity(u1, g)
        df = df.append({'Word':n, 'Cosine_similarity_with_g':cosine_similarity_score}, ignore_index=True)
    else:
        printp('One or all the words in %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%n)
print('\n\n')
printp('List of names and their similarities with constructed vector g:')
display(df)

<p style='color:firebrick;font-weight:bold;font-family:verdana'>Below is the resultant bias vector of vector man and vector woman </p>

[ 0.087144   -0.2182      0.40986     0.03922     0.1032     -0.94165
  0.06042    -0.32988    -0.46144     0.35962    -0.31102     0.86824
 -0.96006    -0.01073    -0.24337    -0.08193     1.02722     0.21122
 -0.695044    0.00222    -0.29106    -0.5053      0.099454   -0.40445
 -0.30181    -0.1355      0.0606      0.07131     0.19245     0.06115
  0.3204     -0.07165     0.13337     0.25068714  0.14293     0.224957
  0.149      -0.048882   -0.12191     0.27362     0.165476    0.20426
 -0.54376     0.271425    0.10245     0.32108    -0.2516      0.33455
  0.04371    -0.01258   ]





<p style='color:firebrick;font-weight:bold;font-family:verdana'>List of names and their similarities with constructed vector g:</p>

,Word,Cosine_similarity_with_g
0,john,0.231634
1,marie,-0.315598
2,sophie,-0.318688
3,ronaldo,0.312448
4,priya,-0.176320
5,rahul,0.169155
6,danielle,-0.243933
7,reza,0.079304
8,katy,-0.283107
9,yasmin,-0.233139


***

**TASK 7: What do you abserve?**

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Observation - <br>
Vector subtraction gives us resultant vector which depicts the difference between the two vectors. For example we substract
word embedded vectors of france and paris. The difference vectors of france and paris is in same space which represents 'is 
capital of' relation. Similarly, when you subtract vectors of woman - man, you get resultant vector g which represents 'is gender' relation. <br>
Here, <br>
1. Male names got positive score and female names got negative score.
2. When we do g = woman - man, and calculate cosine similarity between g and words in name_list, the similarity will be negative for male names and positive for female names. <br>
3. When we do g = man - woman, and calculate cosine similarity between g and words in name_list, the similarity will be positve
for male names and negative for female names. The similarity sign changes here because we altered subtraction sequence and so
resultant vector changed the direction.<br>

Thus, we can conclude that these words carry gender bias.<br>
</p>
</div>

<br>

**Task 8: Compute and print the similarity between g and the words in word_list:**

***

In [20]:
# Lets recompute and use 1st g, which is woman - man, from Task 6
g = np.subtract(word_to_vec_map.get('woman') , word_to_vec_map.get('man'))

df = pd.DataFrame(columns = ['Word','Cosine_similarity_with_g'])

word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']


for n in word_list:
    if checkIfWordExistsInDictionary(n):
        u2 = word_to_vec_map.get(n) 
        cosine_similarity_score= cosine_similarity(u2, g)
        df = df.append({'Word':n, 'Cosine_similarity_with_g':cosine_similarity_score}, ignore_index=True)
    else:
        printp('One or all the words in %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%n)
printp('Other words and their similarities with constructed vector g:')
display(df)

<p style='color:firebrick;font-weight:bold;font-family:verdana'>Other words and their similarities with constructed vector g:</p>

,Word,Cosine_similarity_with_g
0,lipstick,0.276919
1,guns,-0.188849
2,science,-0.060829
3,arts,0.008189
4,literature,0.064725
5,warrior,-0.209202
6,doctor,0.118953
7,tree,-0.070894
8,receptionist,0.330779
9,technology,-0.131937


***

**TASK 9: What do you abserve?**

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Observation - <br>
We can see from above results that some words have positive cosine similarity and others have negative cosine similarity. This
is becuase to compute this similarity we used bias vector g representing space 'is gender'. The resultant output says that
guns is more similar/closer to man whereas receptionist is more similar/closer to woman. <br>

Thus, we can conclude that these words carry gender specific bias.<br>
</p>
</div>

<br>


We'll see below how to reduce the bias of these vectors, using an algorithm due to [Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520). Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. You will have to treat these two type of words differently when debiasing.

# Neutralize bias for non-gender specific words 

The figure below should help you visualize what neutralizing does. If you're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction $g$, and the remaining 49 dimensions, which we'll call $g_{\perp}$. In linear algebra, we say that the 49 dimensional $g_{\perp}$ is perpendicular (or "orthogonal") to $g$, meaning it is at 90 degrees to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 

Even though $g_{\perp}$ is 49 dimensional, given the limitations of what we can draw on a screen, we illustrate it using a 1 dimensional axis below. 


![](images/neutral.png)


<caption><center> <b>Figure 2</b>: The word vector for "receptionist" represented before and after applying the neutralize operation. </center></caption>

**TASK 10**: Implement `neutralize()` to remove the bias of words such as "receptionist" or "scientist". Given an input embedding $e$, you can use the following formulas to compute $e^{debiased}$: 


\begin{equation}
e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}
\end{equation}

\begin{equation}
e^{debiased} = e - e^{bias\_component}\tag{3}
\end{equation}

If you are an expert in linear algebra, you may recognize $e^{bias\_component}$ as the projection of $e$ onto the direction $g$. If you're not an expert in linear algebra, don't worry about this.

<!-- 
**Reminder**: a vector $u$ can be split into two parts: its projection over a vector-axis $v_B$ and its projection over the axis orthogonal to $v$:
$$u = u_B + u_{\perp}$$
where : $u_B = $ and $ u_{\perp} = u - u_B $
!--> 
***

In [21]:
# TASK 10
# GRADED neutralize

def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    e1 = word_to_vec_map.get(word)
    e_dot_g = np.dot(e1, g)
    sq_g = np.square(g) 
    g_length_squared = np.sum(sq_g)
    e_bias_component = np.multiply(np.divide(e_dot_g, g_length_squared), g)
    e_debiased = e1 - e_bias_component 
    return e_debiased
    

In [22]:
e = "receptionist"
if checkIfWordExistsInDictionary(e):
    cs_before = cosine_similarity(word_to_vec_map.get(e), g)
    printp("cosine similarity between " + e + " and g, before neutralizing: %f"%cs_before)

    e_debiased = neutralize(e, g, word_to_vec_map)
    cs_after = cosine_similarity(e_debiased, g)
    printp("cosine similarity between " + e + " and g, after neutralizing: %s"%str(cs_after))
else:
    printp('One or all the words in %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%n)

<p style='color:firebrick;font-weight:bold;font-family:verdana'>cosine similarity between receptionist and g, before neutralizing: 0.330779</p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>cosine similarity between receptionist and g, after neutralizing: -2.920516166121757e-17</p>

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Observation<br>
We can see after debiasing 'receptionist' it's cosine similarity became almost 0. This shows that 'receptionist' vector is now orthogonal to gender bias vector g. We almost reduced the gender bias associated with 'receptionist'
</p>
</div>

<br>

# Equalization algorithm for gender-specific words

Next, lets see how debiasing can also be applied to word pairs such as "actress" and "actor." Equalization is applied to pairs of words that you might want to have differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equi-distant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other word that has been neutralized. In pictures, this is how equalization works: 


![](images/equalize10.png)



The derivation of the linear algebra to do this is a bit more complex. (See Bolukbasi et al., 2016 for details.) But the key equations are: 

\begin{equation}
\mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}
\end{equation}

\begin{equation}
\mu_{B} = \frac {\mu \cdot bias\_axis}{||bias\_axis||_2^2} *bias\_axis
\tag{5}
\end{equation}

\begin{equation}
\mu_{\perp} = \mu - \mu_{B} \tag{6}
\end{equation}

\begin{equation}
e_{w1B} = \frac {e_{w1} \cdot bias\_axis}{||bias\_axis||_2^2} *bias\_axis
\tag{7}
\end{equation}

\begin{equation}
e_{w2B} = \frac {e_{w2} \cdot bias\_axis}{||bias\_axis||_2^2} *bias\_axis
\tag{8}
\end{equation}

\begin{equation}
e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B||} \tag{9}
\end{equation}

\begin{equation}
e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B||} \tag{10}
\end{equation}

\begin{equation}
e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}
\end{equation}

\begin{equation}
e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}
\end{equation}

**TASK 11**: Implement the function below. Use the equations above to get the final equalized version of the pair of words. Good luck!

***

In [23]:
# TASK 11
# GRADED equalize

def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    w1, w2 = pair[0], pair[1]                                           # Unwrapping words
    
    e1 = word_to_vec_map.get(w1)                                        # Fetching word embedded vector from dict
    e2 = word_to_vec_map.get(w2)                                        # Fetching word embedded vector from dict
    
    # STEP 1
    mu = (e1 + e2) / 2                                                  # Calculating mu
    
    # STEP 2
    dot_mb = np.dot(mu, bias_axis)                                      # Calculating dot product of mu and bias_axis
    sq_bias_axis = np.square(bias_axis) 
    bias_axis_length_squared = np.sum(sq_bias_axis)                     # Calculation squared norm of bias axis
    
    muB = (dot_mb / bias_axis_length_squared) * bias_axis               # Calculating muB
    
    # STEP 3
    mu_orth = mu - muB                                                  # Calculating resultant orthogonal mu
    
    # Step 4
    dot_e1b = np.dot(e1, bias_axis)  
    e1B = (dot_e1b / bias_axis_length_squared) * bias_axis
    
    # Step 5
    dot_e2b = np.dot(e2, bias_axis)  
    e2B = (dot_e2b / bias_axis_length_squared) * bias_axis
    
    # Step 6
    sq_mu_orth = np.square(mu_orth) 
    mu_orth_length_squared = np.sum(sq_mu_orth) 
    denominator =  np.sum(e1 - mu_orth - muB)
    right_side = np.sqrt(abs(1 - mu_orth_length_squared))
    left_side = (e1B - muB) / denominator
    e1B_corrected = right_side * left_side                             
    
    # Step 7
    denominator1 =  np.sum(e2 - mu_orth - muB)
    left_side1 = (e2B - muB) / denominator1
    e2B_corrected = right_side * left_side1                             
    
    # STEP 8
    e_1 = e1B_corrected + mu_orth                                      # Equalized w1
    e_2 = e2B_corrected + mu_orth                                      # Equalized w2
    
    return e_1, e_2
    

In [24]:
dictwords = {'man':'woman','lion':'lioness','cat':'tomcat','hen':'rooster','tiger':'tigress'}
for k in dictwords.keys():
    m = k
    w = dictwords[k]
    if checkIfWordExistsInDictionary(m) and checkIfWordExistsInDictionary(w):
        printp('FOR %s AND %s'%(m,w))
        printp("before equalizing:")

        man = cosine_similarity(word_to_vec_map[m], g)
        printg("cosine_similarity("+m+", gender) = %s"%str(man))

        woman = cosine_similarity(word_to_vec_map[w], g)
        printg("cosine_similarity("+w+", gender) = %s "%str(woman))

        e1, e2 = equalize((m, w), g, word_to_vec_map)                           # Equalizing woman, man around gender
        printp("after equalizing:")

        e1g = cosine_similarity(e1, g)
        printg("cosine_similarity("+m+"_equalized, gender) = %s"%str(e1g))

        e2g = cosine_similarity(e2, g)
        printg("cosine_similarity("+w+"_equalized, gender) = %s"%str(e2g))
        print('\n')
    else:
        printp('One or all the words in %s %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%(m,w))
        print('\n')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>FOR man AND woman</p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>before equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(man, gender) = -0.1171109576533683</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(woman, gender) = 0.3566661884627037 </p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>after equalizing:</p>

...

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(tiger, gender) = -0.2715448478765569</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(tigress, gender) = 0.159232961574708 </p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>after equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(tiger_equalized, gender) = 0.7483838947967763</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(tigress_equalized, gender) = 0.7483838947967763</p>

**TASK 12: What do you abserve?**

***

PLEASE ANSWER HERE!


***

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Observation<br>
Before equalization the gender specific words were not equally similar w.r.t gender bias vector space. After equalization of these words, they are now equally similar w.r.t gender bias vector space. We should see what effect this equalization had on similarity of other words like 'receptionist' on which has gender bias. 
</p>
</div>

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Now, let's see what effect this equalization had on receptionist !!!
</p>
</div>

In [25]:
m = 'man'
w = 'woman'
r = 'receptionist'
if checkIfWordExistsInDictionary(m) and checkIfWordExistsInDictionary(w) and checkIfWordExistsInDictionary(r):
    printp("before equalizing:")
    mr = cosine_similarity(word_to_vec_map[m], word_to_vec_map[r])
    printg("cosine_similarity("+m+", receptionist) = %s"%str(mr))

    woman = cosine_similarity(word_to_vec_map[w], word_to_vec_map[r])
    printg("cosine_similarity("+w+", receptionist) = %s "%str(woman))

    e1, e2 = equalize((m, w), g, word_to_vec_map)                        # Equalizing woman, man around gender
    printp("after equalizing:")
    mr = cosine_similarity(e1, word_to_vec_map[r])
    printg("cosine_similarity("+m+", receptionist) = %s"%str(mr))

    woman = cosine_similarity(e2, word_to_vec_map[r])
    printg("cosine_similarity("+w+", receptionist) = %s "%str(woman))
else:
    printp('One or all the words in %s %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%(m,w))

<p style='color:firebrick;font-weight:bold;font-family:verdana'>before equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(man, receptionist) = 0.3104243175079627</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(woman, receptionist) = 0.4464364858934247 </p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>after equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(man, receptionist) = 0.375515758178286</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(woman, receptionist) = 0.3755157581782861 </p>

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Now, let's take better estimation of g by averaging gender spaces of set of words.
</p>
</div>

In [26]:
m = 'man'
w = 'woman'
r = 'receptionist'

dictOfWords = {'woman':'man', 'girl':'boy', 'female':'male', 'mother':'father', 'grandmother':'grandfather', 'sister':'brother'\
               ,'wife':'husband', 'lady':'lord', 'queen':'king', 'princess':'prince','aunty':'uncle','niece':'nephew'}

g1 = 0

for a in dictOfWords.keys():    
    w1 = a
    w2 = dictOfWords[a]
    if checkIfWordExistsInDictionary(w1) and checkIfWordExistsInDictionary(w2):
        e1 = word_to_vec_map[w1]
        e2 = word_to_vec_map[w2]
        g1 = g1 + (e1 - e2)
    else:
        printp('One or all the words in %s %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%(m,w))
        print(w)
        print(a)
        print(dictOfWords[a])

g = g1 / len(dictOfWords)                                                   # Average gender bais vector

printp('Lets check distance of man, woman and receptionist with new bias vector')
printp("before equalizing:")
mr = cosine_similarity(word_to_vec_map[m], g)
printg("cosine_similarity("+m+", gender bias) = %s"%str(mr))

woman = cosine_similarity(word_to_vec_map[w], g)
printg("cosine_similarity("+w+", gender bias) = %s "%str(woman))

e1, e2 = equalize((m, w), g, word_to_vec_map)                              # Equalizing woman, man around gender
printp("after equalizing:")
mr = cosine_similarity(e1, g)
printg("cosine_similarity("+m+", gender bias) = %s"%str(mr))

woman = cosine_similarity(e2, g)
printg("cosine_similarity("+w+", gender bias) = %s "%str(woman))

mr = cosine_similarity(e1, word_to_vec_map[r])
printg("cosine_similarity("+m+", receptionist) = %s"%str(mr))

woman = cosine_similarity(e2, word_to_vec_map[r])
printg("cosine_similarity("+w+", receptionist) = %s "%str(woman))

<p style='color:firebrick;font-weight:bold;font-family:verdana'>Lets check distance of man, woman and receptionist with new bias vector</p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>before equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(man, gender bias) = -0.13383132990296484</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(woman, gender bias) = 0.22047402726555493 </p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>after equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(man, gender bias) = 0.9833248103943684</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(woman, gender bias) = 0.9833248103943684 </p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(man, receptionist) = 0.388099587001532</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(woman, receptionist) = 0.3880995870015321 </p>

<br>

<br>

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Age Biased Words
</p>
</div>

In [27]:
w1 = 'young'
w2 = 'old'

u = word_to_vec_map.get(w1) 
v = word_to_vec_map.get(w2) 
age = np.subtract(u, v)

printp('Below is the resultant age bias vector of vector young and vector old ')
print(age)

df = pd.DataFrame(columns = ['Word','Cosine_similarity_with_age'])

name_list = ['manager', 'assistant']

for n in name_list:
    if checkIfWordExistsInDictionary(n):
        u1 = word_to_vec_map.get(n) 
        cosine_similarity_score= cosine_similarity(u1, age)
        df = df.append({'Word':n, 'Cosine_similarity_with_age':cosine_similarity_score}, ignore_index=True)
    else:
        printp('One or all the words in %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%n)
print('\n\n')
printp('List of names and their similarities with constructed vector age:')
display(df)

<p style='color:firebrick;font-weight:bold;font-family:verdana'>Below is the resultant age bias vector of vector young and vector old </p>

[ 0.09793   -0.18399    0.299309  -0.72784    0.10094   -0.11091
  0.2189    -0.125991  -0.52477    0.33089    0.213981   0.54913
  0.656614   0.10779   -0.26219   -0.694882   1.05802   -0.12921
 -0.51308   -0.724884   0.1522     0.15397    1.38748    0.31858
  0.28643    0.7563    -0.20041   -1.20299   -0.53337   -0.41608
  0.071      1.30329    0.190959  -1.43544   -0.3196    -0.20773
 -0.29931   -0.55679   -0.5899578 -0.29469   -0.18698    0.00712
  0.00762   -0.342021   0.75049   -0.73536   -0.22936   -0.07647
 -0.18104    1.03046  ]





<p style='color:firebrick;font-weight:bold;font-family:verdana'>List of names and their similarities with constructed vector age:</p>

,Word,Cosine_similarity_with_age
0,manager,-0.024268
1,assistant,0.002551


<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Observation - <br>
From above results we can see that 'manager' is more closer to 'old' whereas 'assistant' is more closer to 'young'. This is not always true in real world scenarios and can thus be considered as biased. Let's equlize 'young' and 'old' around age.
</p>
</div>

In [28]:
dictwords = {'young':'old'}
for k in dictwords.keys():
    m = k
    w = dictwords[k]
    if checkIfWordExistsInDictionary(m) and checkIfWordExistsInDictionary(w):
        printp('FOR %s AND %s'%(m,w))
        printp("before equalizing:")

        manager = cosine_similarity(word_to_vec_map[m], age)
        printg("cosine_similarity("+m+", gender) = %s"%str(manager))

        assit = cosine_similarity(word_to_vec_map[w], age)
        printg("cosine_similarity("+w+", gender) = %s "%str(assit))

        e1, e2 = equalize((m, w), age, word_to_vec_map)                   # Equalizing young, old around age
        printp("after equalizing:")

        e1g = cosine_similarity(e1, age)
        printg("cosine_similarity("+m+"_equalized, gender) = %s"%str(e1g))

        e2g = cosine_similarity(e2, age)
        printg("cosine_similarity("+w+"_equalized, gender) = %s"%str(e2g))
        print('\n')
    else:
        printp('One or all the words in %s %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%(m,w))
        print('\n')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>FOR young AND old</p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>before equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(young, gender) = 0.3464061089377554</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(old, gender) = -0.42129240017943104 </p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>after equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(young_equalized, gender) = -0.9136070807663154</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(old_equalized, gender) = -0.9136070807663156</p>

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Now, let's see if manager is equidistance from young and old
</p>
</div>

In [29]:
m = 'young'
w = 'old'
r = 'manager'
if checkIfWordExistsInDictionary(m) and checkIfWordExistsInDictionary(w) and checkIfWordExistsInDictionary(r):
    printp("before equalizing:")
    mr = cosine_similarity(word_to_vec_map[m], word_to_vec_map[r])
    printg("cosine_similarity("+m+", manager) = %s"%str(mr))

    woman = cosine_similarity(word_to_vec_map[w], word_to_vec_map[r])
    printg("cosine_similarity("+w+", manager) = %s "%str(woman))

    e1, e2 = equalize((m, w), age, word_to_vec_map)                       # Equalizing young, old around age
    printp("after equalizing:")
    mr = cosine_similarity(e1, word_to_vec_map[r])
    printg("cosine_similarity("+m+", manager) = %s"%str(mr))

    woman = cosine_similarity(e2, word_to_vec_map[r])
    printg("cosine_similarity("+w+", manager) = %s "%str(woman))
else:
    printp('One or all the words in %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%n)

<p style='color:firebrick;font-weight:bold;font-family:verdana'>before equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(young, manager) = 0.43696476970959663</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(old, manager) = 0.4408017196155107 </p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>after equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(young, manager) = 0.21521080327992223</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(old, manager) = 0.21521080327992234 </p>

<br>

<br>

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Religion Biased Words
</p>
</div>

In [30]:
w1 = 'christian'
w2 = 'atheist'

u = word_to_vec_map.get(w1) 
v = word_to_vec_map.get(w2) 
religion = np.subtract(u, v)

printp('Below is the resultant religion bias vector of vector christian and vector atheist ')
print(religion)

df = pd.DataFrame(columns = ['Word','Cosine_similarity_with_religion'])

name_list = ['john', 'rahul']

for n in name_list:
    if checkIfWordExistsInDictionary(n):
        u1 = word_to_vec_map.get(n) 
        cosine_similarity_score= cosine_similarity(u1, religion)
        df = df.append({'Word':n, 'Cosine_similarity_with_religion':cosine_similarity_score}, ignore_index=True)
    else:
        printp('One or all the words in %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%n)
print('\n\n')
printp('List of names and their similarities with constructed vector religion:')
display(df)

<p style='color:firebrick;font-weight:bold;font-family:verdana'>Below is the resultant religion bias vector of vector christian and vector atheist </p>

[ 0.712627   0.151391  -1.49459   -0.73336    0.75447   -0.28789
 -0.83355   -0.41619    0.62702   -0.4091     0.18549   -0.2057
 -1.18275   -0.75755   -0.334219  -0.44822    0.74648   -1.11407
 -0.514853  -1.01931    0.31253    0.1217    -0.690095   0.33906
 -1.02617   -0.20596   -0.44337   -0.31579   -0.86402   -0.242325
  2.02514    1.16612   -0.0071    -0.22619    0.63633    0.44659
 -0.70565    0.07801    0.1968311  0.93174    0.54729    1.56226
 -0.84974   -1.407781   0.167824   1.07936    0.80377    0.2047
 -0.92335   -0.585644 ]





<p style='color:firebrick;font-weight:bold;font-family:verdana'>List of names and their similarities with constructed vector religion:</p>

,Word,Cosine_similarity_with_religion
0,john,0.412841
1,rahul,-0.007393


<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Observation - <br>
From above results we can see that 'john' is more closer to 'christian' whereas 'rahul' is more closer to 'atheist'. This is not always true in real world scenarios and can thus be considered as biased. Let's equlize 'christian' and 'atheist' around religion.
</p>
</div>

In [31]:
dictwords = {'christian':'atheist'}
for k in dictwords.keys():
    m = k
    w = dictwords[k]
    if checkIfWordExistsInDictionary(m) and checkIfWordExistsInDictionary(w):
        printp('FOR %s AND %s'%(m,w))
        printp("before equalizing:")

        manager = cosine_similarity(word_to_vec_map[m], religion)
        printg("cosine_similarity("+m+", religion) = %s"%str(manager))

        assit = cosine_similarity(word_to_vec_map[w], religion)
        printg("cosine_similarity("+w+", religion) = %s "%str(assit))

        e1, e2 = equalize((m, w), religion, word_to_vec_map)                 # Equalizing christian and atheist around religion
        printp("after equalizing:")

        e1g = cosine_similarity(e1, religion)
        printg("cosine_similarity("+m+"_equalized, religion) = %s"%str(e1g))

        e2g = cosine_similarity(e2, religion)
        printg("cosine_similarity("+w+"_equalized, religion) = %s"%str(e2g))
        print('\n')
    else:
        printp('One or all the words in %s %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%(m,w))
        print('\n')

<p style='color:firebrick;font-weight:bold;font-family:verdana'>FOR christian AND atheist</p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>before equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(christian, religion) = 0.5872275887463786</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(atheist, religion) = -0.4623048412507701 </p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>after equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(christian_equalized, religion) = -0.7668560263802137</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(atheist_equalized, religion) = -0.7668560263802137</p>

<div style="background-color:#B5C689"  class="alert alert-block alert-info">
<p style="color:#0C374D; fontsize:80px">
Now, let's see if rahul is equidistance from christian and atheist
</p>
</div>

In [32]:
m = 'christian'
w = 'atheist'
r = 'rahul'
if checkIfWordExistsInDictionary(m) and checkIfWordExistsInDictionary(w) and checkIfWordExistsInDictionary(r):
    printp("before equalizing:")
    mr = cosine_similarity(word_to_vec_map[m], word_to_vec_map[r])
    printg("cosine_similarity("+m+", rahul) = %s"%str(mr))

    woman = cosine_similarity(word_to_vec_map[w], word_to_vec_map[r])
    printg("cosine_similarity("+w+", rahul) = %s "%str(woman))

    e1, e2 = equalize((m, w), religion, word_to_vec_map)                 # Equalizing christian and atheist around religion
    printp("after equalizing:")
    mr = cosine_similarity(e1, word_to_vec_map[r])
    printg("cosine_similarity("+m+", rahul) = %s"%str(mr))

    woman = cosine_similarity(e2, word_to_vec_map[r])
    printg("cosine_similarity("+w+", rahul) = %s "%str(woman))
else:
    printp('One or all the words in %s %s do not exists in the mapping dictionary. Check if the words are correctly spelled'%(m,w))

<p style='color:firebrick;font-weight:bold;font-family:verdana'>before equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(christian, rahul) = 0.02795644159168636</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(atheist, rahul) = 0.038799801860380304 </p>

<p style='color:firebrick;font-weight:bold;font-family:verdana'>after equalizing:</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(christian, rahul) = 0.03127916471614878</p>

<p style='color:green;font-weight:bold;font-family:monaco'>cosine_similarity(atheist, rahul) = 0.03127916471614876 </p>

<br>

Please feel free to play with the input words in the cell above, to apply equalization to other pairs of words. 

These debiasing algorithms are very helpful for reducing bias, but are not perfect and do not eliminate all traces of bias. For example, one weakness of this implementation was that the bias direction $g$ was defined using only the pair of words _woman_ and _man_. As discussed earlier, if $g$ were defined by computing $g_1 = e_{woman} - e_{man}$; $g_2 = e_{mother} - e_{father}$; $g_3 = e_{girl} - e_{boy}$; and so on and averaging over them, you would obtain a better estimate of the "gender" dimension in the 50 dimensional word embedding space. Feel free to play with such variants as well.  
                     

# Congratulations!

You've come to the end of this assignment, and have seen a lot of the ways that word vectors can be used as well as  modified. Here are the main points you should remember:

- Cosine similarity a good way to compare similarity between pairs of word vectors. (Though L2 distance works too.) 
- For NLP applications, using a pre-trained set of word vectors from the internet is often a good way to get started. 
- Bias in data is an important problem.
- Neutralize and equalize allow to reduce bias in the data.

Congratulations on finishing this notebook! 



# References
- The debiasing algorithm is from Bolukbasi et al., 2016, [Man is to Computer Programmer as Woman is to
Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)
- The GloVe word embeddings were due to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (https://nlp.stanford.edu/projects/glove/)
